In [1]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [2]:
monkey.patch_all(thread=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = True
server.serve_forever()
#http://127.0.0.2:5000/

127.0.0.1 - - [2020-02-25 09:16:39] "GET / HTTP/1.1" 200 6818 0.177096
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 304 211 0.096294
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/css/template.css HTTP/1.1" 304 210 0.001052
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/css/mycss.css HTTP/1.1" 304 209 0.002003
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 304 210 0.017931
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 304 211 0.001001
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 304 210 0.002012
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/js/template.js HTTP/1.1" 304 209 0.001504
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/fontawesome-free/webfonts/fa-solid-900.woff2 HTTP/1.1" 304 211 0.006015
127.0.0.1 - - [2020-02-25 09:16:40] "GET /static/vendor/fontawesome-free/webfont

KeyboardInterrupt: 

In [ ]:
import sys
sys.path.insert(1, r'C:\Users\Kart\jupyter_projects')
import mygit as mg

In [ ]:
folder=r'C:\Users\Kart\jupyter_projects\webtrade'
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\src")

In [ ]:
from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_sectors(init_db_manager())

In [ ]:
a=job_sl_bonds(init_db_manager())

In [ ]:
a=init_db_manager().get_table('smartlabbondsrus',result='matrix')

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

In [ ]:
from external_sources import smartlab
import pandas as pd
a=smartlab().bonds_info()